In [1]:
from music21 import converter, instrument, note, chord
import glob
from keras.utils import np_utils
import pickle
import numpy as np

Using TensorFlow backend.
C:\Users\SRINIVAS\Anaconda3\envs\innomatics\lib\site-packages\pydot.py:18: UserWarning: Couldn't import dot_parser, loading of dot files will not be possible.
  "Couldn't import dot_parser, "


In [2]:
#load the notes used to train the model
with open('notes', 'rb') as filepath:
    notes = pickle.load(filepath)

In [3]:

n_vocab=len(set(notes))

sequence_length = 100
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

In [4]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation
from keras.callbacks import ModelCheckpoint

In [5]:
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
model.load_weights('weights_folder/my_weights-improvement=168-0.1347-bigger.hdf5')

In [12]:
model.save('music_generator.h5')
import netron
netron.start('music_generator.h5')

Serving 'music_generator.h5' at http://localhost:8080


In [ ]:
# pick a random sequence from the input as a starting point for the prediction
start = np.random.randint(0, len(network_input)-1)
start

int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

pattern = network_input[start]
prediction_output = []

pattern=list(pattern)
pattern

for note_index in range(500):
    prediction_input=np.reshape(pattern,(1,len(pattern),1))
    prediction_input=prediction_input/float(n_vocab)
    
    prediction=model.predict(prediction_input,verbose=0)
    
    index=np.argmax(prediction)
    result=int_to_note[index]
    
    prediction_output.append(result)
    
    pattern.append(index)
    
    pattern=pattern[1:len(pattern)]

prediction_output

from music21 import instrument, note, stream, chord

offset = 0
output_notes = []

# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    # increase offset each iteration so that notes do not stack
    offset += 0.5

midi_stream = stream.Stream(output_notes)

midi_stream.write('midi', fp='new_test_output_168.mid')


prediction_output